In [1]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"    # 日志格式化输出
DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"                        # 日期格式
fp = logging.FileHandler('esio.log')
fs = logging.StreamHandler()
logging.basicConfig(level=logging.WARNING, format=LOG_FORMAT, datefmt=DATE_FORMAT, handlers=[fs])    # 调用

import pandas as pd
from pandasticsearch import DataFrame, Select
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, parallel_bulk
from tqdm import tqdm

from sklearn import datasets

from tool.sink import ESSink

In [2]:
df = datasets.load_iris()
df = pd.DataFrame(df.data, columns=['x1','x2','x3','x4'])
df.head(4)

,x1,x2,x3,x4
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


In [3]:
es_localhost = "192.168.3.248:9200"
es_ysyp_test = "115.29.34.243:9200"
unsername = 'elastic'
password = 'superzsh123'

es = Elasticsearch(
    [es_ysyp_test],
    http_auth=(unsername, password)
)

In [4]:
%%time
i2i = ESSink(df, es)
i2i.to_es("i2i")

CPU times: user 14.8 ms, sys: 2.37 ms, total: 17.2 ms
Wall time: 329 ms


In [5]:
%%time
# df = DataFrame.from_es(url='http://115.29.34.243:9200', username=unsername, password=password, index='index_recall_pool', doc_type='type_recall_i2i', compat=5).to_pandas()
result_dict = es.search(index="i2i", body={"query": {"match_all": {}}, "size":1000})
df = Select.from_dict(result_dict).to_pandas()

CPU times: user 3.18 ms, sys: 1.21 ms, total: 4.4 ms
Wall time: 31.6 ms


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
_id       150 non-null object
_index    150 non-null object
_score    150 non-null float64
_type     150 non-null object
x1        150 non-null float64
x2        150 non-null float64
x3        150 non-null float64
x4        150 non-null float64
dtypes: float64(5), object(3)
memory usage: 9.5+ KB
